### 鸭子类型
只需要按照预定行为实现对象所需要的方法即可,比如repr()和str()背后支持的__repr__,`__str__`.还有两个特殊方法__bytes__和__format__可以为对象提供其他表示形式,bytes()函数调用它获取对象的字节序列表示形式,format()和str.format()通过调用obj.`__format__`(format_spec)以特殊的格式化代码显示对象的字符串表示形式

### 备选构造函数
- classmethod:最常见用途是用来定义备选构造函数,第一个参数是类本身
- staticmethod:静态方法就是普通方法,只是碰巧位于类的定义体中,而不是在模块层

### 支持位匹配模式
需要加`__match_args_=(filed1,filed2)`的类属性,其中顺序上位匹配模式的顺序

### python私有属性和‘受保护’的属性
python不能像java那样使用private修饰符创建私有属性,但是它有有一个简单的机制,能避免类意外覆盖.在实例属性名前面加两个下划线,python会把属性名存入实例属性__dict__中,而且会在前面加一个下划线和类名,名称改写

使用单个下划线的只是一种约定,python不会对它做处理

In [6]:
import math
import array
class Vector:
    typecode='d'

    def __init__(self,x,y):
        self.__x=float(x)
        self.__y=float(y)
    
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    #abs()方法调用
    def __abs__(self):
        return math.sqrt(self.__x**2+self.__y**2)
    
    def __iter__(self):
        return (val for val in (self.__x,self.__y) )
    
    def __eq__(self, other):
        return tuple(self)==tuple(other)
    
    def __repr__(self):
        class_name=type(self).__name__
        return f'{class_name} {self.__x} {self.__y}'
    
    # 内置函数bytes
    def __bytes__(self):
        return (bytes([ord(self.typecode)])) + bytes(array.array(self.typecode,self))
    
    def __format__(self,formar_spec=''): 
        components=(format(c,formar_spec) for c in self)
        return '({} {})'.format(*components)

    @classmethod
    def from_bytes(cls,octets):
        type_code=chr(octets[0])
        memv=memoryview(octets[1:]).cast(type_code)
        return cls(*memv)

    # 可hash对象不能改变
    def __hash__(self) -> int:
        return hash((self.__x,self.__y))

v1=Vector(3,4) 
abs(v1)

t=tuple(v1)
print(t)

bv=bytes(v1)
v2=Vector.from_bytes(bv)
print(v2)

print(format(v2))
hash(v1)

(3.0, 4.0)
Vector 3.0 4.0
(3.0 4.0)


1079245023883434373

### 使用__slots__节约空间
python把各个实例的属性都存储在一个名为__dict__字典中.可以通过定义一个名为__slots__的类属性,以序列的形式存储属性名称,那么python将使用其他模型存储实例属性

注意问题
- 每个子类都要重新声明__slots__属性,以防止子类实例有__dict__属性
- 实例只能拥有__slots__列出的属性,除非把__dict__属性加入__slots__中(这样也失效)
- 有__slots__的类不能使用@cached_propety装饰器,除非把__dict__加入__slots__中
- 如果不把__weakref__加入__slots__中,那么实例就不能作为弱引用的目标

In [14]:
class Pixel:
    __slots__=('x','y')

    def __init__(self,x,y):
        self.x=x
        self.y=y

p=Pixel(1,2)

# 声明__slots__以后实例就没有__dict__属性,如果设置没有属性将会ArrtibuteError
# p.__dict__ 

class ColorPixel(Pixel):
    # 在子类申请__slots_,子类将没有__dict__
    #__slots__=()
    def __init__(self,x,y,z):
        Pixel.__init__(self,x,y)
        self.z=z

# 
c=ColorPixel(1,2,3)
print(c.x,c.y,c.z)
print(c.__dict__)

1 2 3
{'z': 3}
